In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. 먼저 현재 작업 디렉토리를 확인합니다:
import os
print(os.getcwd())
# 2. 필요한 경우 작업 디렉토리를 변경합니다:
os.chdir('/content/drive/MyDrive/Colab Notebooks/아이펠/DLThon/')#←('/content')  # 코랩의 기본 디렉토리로 변경
# 3. 그 다음 상대 경로를 사용하여 파일을 불러올 수 있습니다:

/content


여기까지 하고 시작

---

In [4]:
# 불용어 리스트 불러오기
def load_stopwords(file_path='weighted_korean_stopwords.txt'):
    with open(file_path, 'r', encoding='utf-8') as f:
        stopwords = [line.strip() for line in f if line.strip()]
    print(f"불용어 {len(stopwords)}개 로드 완료")
    return stopwords


In [5]:
import os
import re
import numpy as np
import pandas as pd
from functools import partial
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizer, ElectraForSequenceClassification#, AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns

# 시드 고정
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True

set_seed(42)

#--------------------------
# 데이터 전처리 함수들
#--------------------------

# 1. 텍스트 정제 함수들
def remove_stopwords(tokens, stopword_list):
    stopword_set = stopword_list if isinstance(stopword_list, set) else set(stopword_list)
    return [token for token in tokens if token not in stopword_set]

def clean_text(text):
    text = re.sub(r"[^\w\s가-힣]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def normalize_repetitions(text, repeat_limit=2):
    # 문자 반복 (예: ㅋㅋㅋㅋ → ㅋㅋ)
    text = re.sub(r'(.)\1{2,}', lambda m: m.group(1) * repeat_limit, text)

    # 음절 반복 (예: 하하하하 → 하하)
    text = re.sub(r'((..))\\1{1,}', lambda m: m.group(1) * repeat_limit, text)

    return text

# 2. 텍스트 정제
def tokenize_and_clean_text(text, stopword_list=None, repeat_limit=2):
    text = normalize_repetitions(text, repeat_limit=repeat_limit)
    text = clean_text(text)
    tokens = text.split()
    if stopword_list:
        tokens = remove_stopwords(tokens, stopword_list)
    return tokens

# 3. 한줄 단위 전처리
def preprocess_conversation_lines(
    text,
    stopwords=None,
    use_silence=False,
    speaker_token="[UTTER]",
    repeat_limit=2
):
    lines = text.strip().split('\n')
    results = []

    for line in lines:
        if not line.strip():
            processed = ["[SILENCE]"] if use_silence else []
        else:
            processed = tokenize_and_clean_text(line, stopword_list=stopwords, repeat_limit=repeat_limit)
            if not processed:
                processed = ["[SILENCE]"] if use_silence else []
            else:
                processed = [speaker_token] + processed if speaker_token else processed

        if processed:
            results.append(" ".join(processed).strip())

    return results

# 4. 여러줄을 한 줄로 flatten 함수
def flatten_utterances(utterance_tokens_list, sep_token=" "):
    return sep_token.join(utterance_tokens_list).strip()

# 5. 전체 전처리 파이프라인
def preprocess(
    text,
    stopwords=None,
    speaker_token="[UTTER]",
    use_silence=True,
    sep_token=" ",
    repeat_limit=2
):
    """
    전체 전처리 통합 함수
    """
    utterance_tokens = preprocess_conversation_lines(
        text,
        stopwords=stopwords,
        use_silence=use_silence,
        speaker_token=speaker_token,
        repeat_limit=repeat_limit
    )
    return flatten_utterances(utterance_tokens, sep_token=sep_token)

# 결측치 및 중복 제거 함수
def clean_dataframe(df, is_train=True):
    df = df.dropna(subset=['text'])
    df = df.drop_duplicates(subset=['text'])
    if is_train:
        df = df.dropna(subset=['class'])
    df = df.reset_index(drop=True)
    return df

# CSV 파일 로드 함수
def load_csv_files(file_list, is_train=True):
    df_list = []

    for file_path in file_list:
        df = pd.read_csv(file_path)

        if is_train:
            if 'conversation' in df.columns:
                df = df.rename(columns={'conversation': 'text'})
        else:
            if 'conversation' in df.columns:
                df = df.rename(columns={'conversation': 'text'})
            if 'class' not in df.columns:
                df['class'] = pd.NA  # test에는 class가 없으므로 NaN 처리

        df_list.append(df)

    return pd.concat(df_list, ignore_index=True)

# 데이터프레임 준비 함수
def prepare_dataset(file_paths, preprocess_func, is_train=True):
    df = load_csv_files(file_paths, is_train=is_train)
    df = clean_dataframe(df, is_train=is_train)
    df['text'] = df['text'].apply(preprocess_func)
    df = clean_dataframe(df, is_train=is_train)
    return df

#--------------------------
# 모델 학습 설정
#--------------------------

# 설정값
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 10  # 5에서 10으로 변경
SCHEDULER_EPOCHS = 5  # 스케줄러용 에폭 (원래 설정 유지)
EARLY_STOPPING_PATIENCE = 2  # 연속 2번의 에폭 동안 성능이 향상되지 않으면 학습 중단
LEARNING_RATE = 2e-5
TRAIN_PATH = "data/train+normal_conversation.csv"
TEXT_COL = "text"#"conversation"x : 편리성 때문에
LABEL_COL = "class"
LABEL_DICT = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
ID_TO_LABEL = {v: k for k, v in LABEL_DICT.items()}
GPU_NUM = 0
TEST_FILE = "data/test.csv"

# 1. 데이터 전처리 및 로드
print("데이터 전처리 및 로드 중...")

# # 불용어 리스트 정의
# stopwords = '이 있 하 것 들 그 되 수 이 보 않 없 나 주 아니 등 같 우리 때 년 가 한 지 오 네 야 아 아니 그럼 내가 너'.split()
# 불용어 리스트 불러오기
stopwords = load_stopwords('weighted_korean_stopwords.txt')

# 전처리 함수 설정 - 불용어 적용
preprocess_fn = partial(
    preprocess,
    stopwords=stopwords,   # 불용어 리스트 사용
    speaker_token="",      # 발화 단위 토큰 사용하지 않음
    sep_token=" ",         # 줄 구분 시 토큰
    use_silence=False,     # [SILENCE] 토큰 사용하지 않음
    repeat_limit=2         # 반복 문자 2개까지 허용
)

# 데이터셋 준비
train_files = [TRAIN_PATH]

# 데이터 전처리 및 저장
print("데이터 전처리 시작...")
train_df = prepare_dataset(train_files, preprocess_fn, is_train=True)
# train_df.to_csv("dataset/train_preprocessed_1.csv", index=False)
# print(f"✅ 전처리된 데이터 저장 완료: {train_df.shape}")
print(train_df.head())

# 라벨 매핑
train_df[LABEL_COL] = train_df[LABEL_COL].map(LABEL_DICT)

# 데이터 분할
train_data, temp_data = train_test_split(train_df, test_size=0.3, random_state=42, stratify=train_df[LABEL_COL])
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=temp_data[LABEL_COL])

print(f"학습 데이터 크기: {len(train_data)}")
print(f"검증 데이터 크기: {len(val_data)}")
print(f"테스트 데이터 크기: {len(test_data)}")

# 클래스별 데이터 개수 확인
print("학습 데이터의 클래스 분포:")
print(train_data[LABEL_COL].value_counts())

# 2. Dataset 클래스 정의
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 3. 데이터 로더 생성
def create_data_loader(df, tokenizer, max_len, batch_size):
    dataset = TextClassificationDataset(
        texts=df[TEXT_COL].values,
        labels=df[LABEL_COL].values,
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=2
    )

# 4. 토크나이저 및 모델 로드
print(f"모델 '{MODEL_NAME}' 로드 중...")
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)
model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_DICT))

# 5. 데이터 로더 생성
train_data_loader = create_data_loader(train_data, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(val_data, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_data, tokenizer, MAX_LEN, BATCH_SIZE)

# 6. 모델 학습 함수
def train_model(model, data_loader, optimizer, scheduler, device):
    model.train()
    losses = []

    progress_bar = tqdm(data_loader, desc="학습 중")

    for batch in progress_bar:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        losses.append(loss.item())
        progress_bar.set_postfix({"loss": f"{np.mean(losses):.4f}"})

    return np.mean(losses)

# 7. 모델 평가 함수
def evaluate_model(model, data_loader, device):
    model.eval()
    losses = []
    predictions = []
    real_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="평가 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
            )

            _, preds = torch.max(outputs.logits, dim=1)

            losses.append(outputs.loss.item())
            predictions.extend(preds.detach().cpu().numpy())
            real_labels.extend(labels.detach().cpu().numpy())

    return np.mean(losses), predictions, real_labels

# 8. 테스트 세트 예측 함수
def predict_test(model, data_loader, device):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="테스트 예측 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.detach().cpu().numpy())

    return predictions

# 9. 혼동 행렬 시각화 함수
def plot_confusion_matrix(cm, classes):
    # 한글 폰트 설정
    import matplotlib.font_manager as fm

    # 시스템에 설치된 폰트 경로 확인
    # 리눅스 환경이면 'NanumGothic'이나 다른 한글 폰트 사용
    # 윈도우라면 'Malgun Gothic' 등 사용
    try:
        # 방법 1: 나눔 글꼴 설치 및 사용
        !apt-get update -qq
        !apt-get install fonts-nanum -qq
        plt.rc('font', family='NanumGothic')
    except:
        try:
            # 방법 2: 한글 레이블을 영어로 변환
            classes = ['Threat', 'Extortion', 'Workplace Harassment', 'Other Harassment', 'Normal Conversation']
        except:
            pass

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('예측값')
    plt.ylabel('실제값')
    plt.title('혼동 행렬')
    plt.savefig('confusion_matrix.png')
    plt.close()

# 10. 학습 메인 함수
def train():
    # GPU 사용 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")
    print(f"사용 중인 디바이스: {device}")
    model.to(device)

    # 옵티마이저 및 스케줄러 설정
    # AdamW 임포트 부분 수정
    # from transformers import AdamW - 이 부분이 오류 발생
    from torch.optim import AdamW  # 이렇게 수정
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)  # correct_bias 파라미터 제거
    total_steps = len(train_data_loader) * SCHEDULER_EPOCHS  # 원래 에폭(5) 기준으로 설정

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # 학습 시작
    best_val_f1 = 0
    patience_counter = 0  # Early Stopping을 위한 카운터

    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch + 1}/{EPOCHS}")

        # 학습
        train_loss = train_model(model, train_data_loader, optimizer, scheduler, device)
        print(f"Train Loss: {train_loss:.4f}")

        # 검증
        val_loss, val_predictions, val_labels = evaluate_model(model, val_data_loader, device)
        val_f1 = f1_score(val_labels, val_predictions, average='macro')

        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Macro F1: {val_f1:.4f}")

        # 최고 성능 모델 저장 및 Early Stopping 로직
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"모델 저장됨: best_model.pth (F1: {val_f1:.4f})")
            patience_counter = 0  # 성능이 향상되었으므로 카운터 초기화
        else:
            patience_counter += 1  # 성능이 향상되지 않았으므로 카운터 증가
            print(f"성능 향상 없음: {patience_counter}/{EARLY_STOPPING_PATIENCE}")

            # Early Stopping 조건 확인
            if patience_counter >= EARLY_STOPPING_PATIENCE:
                print(f"\nEarly Stopping! {EARLY_STOPPING_PATIENCE}번의 에폭 동안 성능 향상 없음.")
                break

    # 최고 성능 모델 로드
    model.load_state_dict(torch.load("best_model.pth"))

    # 테스트 세트 평가
    test_loss, test_predictions, test_labels = evaluate_model(model, test_data_loader, device)
    test_f1 = f1_score(test_labels, test_predictions, average='macro')

    print("\n테스트 결과:")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Macro F1: {test_f1:.4f}")

    # 분류 보고서 출력
    class_names = [ID_TO_LABEL[i] for i in range(len(LABEL_DICT))]
    print("\n분류 보고서:")
    print(classification_report(test_labels, test_predictions, target_names=class_names))

    # 혼동 행렬 시각화
    cm = confusion_matrix(test_labels, test_predictions)
    plot_confusion_matrix(cm, class_names)

    return model

# 11. 실제 테스트 데이터 예측 및 제출 파일 생성
def predict_and_save():
    # 테스트 파일 로드
    test_data = pd.read_csv(TEST_FILE)

    # 텍스트 전처리 적용 - 학습 데이터와 동일한 방식으로 전처리
    # 줄 바꿈 표시 X, 불용어 X 설정
    test_data['processed_text'] = test_data['text'].apply(
        lambda x: preprocess(
            x,
            stopwords=stopwords,   # 불용어 리스트 사용
            speaker_token="",      # 발화 단위 토큰 사용하지 않음
            sep_token=" ",         # 줄 구분 시 토큰
            use_silence=False,     # [SILENCE] 토큰 사용하지 않음
            repeat_limit=2         # 반복 문자 2개까지 허용
        )
    )

    # Dataset 생성
    class TestDataset(Dataset):
        def __init__(self, texts, tokenizer, max_len):
            self.texts = texts
            self.tokenizer = tokenizer
            self.max_len = max_len

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, idx):
            text = str(self.texts[idx])

            encoding = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                return_token_type_ids=True,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )

            return {
                'text': text,
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'token_type_ids': encoding['token_type_ids'].flatten(),
            }

    # 테스트 데이터 로더 생성
    test_dataset = TestDataset(
        texts=test_data['processed_text'].values,
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )

    test_data_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=2
    )

    # GPU 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")

    # 최고 성능 모델 로드
    model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_DICT))
    model.load_state_dict(torch.load("best_model.pth"))
    model.to(device)

    # 예측
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(test_data_loader, desc="테스트 예측 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.detach().cpu().numpy())

    # 예측 결과 라벨로 변환 (숫자로 유지)
    label_predictions = [str(p) for p in predictions]

    # 제출 파일 생성
    submission = pd.DataFrame({
        'idx': test_data['idx'],
        'class': label_predictions
    })

    submission.to_csv('data/submission.csv', index=False)
    print(f"제출 파일 생성 완료: data/submission.csv")

# 메인 실행
if __name__ == "__main__":
    # 모델 학습
    model = train()

    # 테스트 데이터 예측 및 제출 파일 생성
    predict_and_save()

데이터 전처리 및 로드 중...
불용어 110개 로드 완료
데이터 전처리 시작...
   idx      class                                               text
0    0      협박 대화  스스로를 죽여달라고 애원하는 것인가 아닙니다 죄송합니다 죽을 거면 혼자 죽지 우리까...
1    1      협박 대화  길동경찰서입니다 9시 40분 마트에 폭발물을 설치할거다 똑바로 들어 한번만 얘기한다...
2    2  기타 괴롭힘 대화  되게 귀여운거 알지 나보다 작은 남자는 첨봤어 그만해 니들 놀리는거 재미없어 지영아...
3    3      갈취 대화  어이 말이야 이리 오라고 일 옷 좋아보인다 돈 있나봐 아니에요 돈 없어요 뒤져서 나...
4    4      갈취 대화  저기요 혹시 날이 너무 뜨겁잖아요 회사에서 선크림 파는데 한 번 손등에 발라보실래요...
학습 데이터 크기: 3477
검증 데이터 크기: 745
테스트 데이터 크기: 746
학습 데이터의 클래스 분포:
class
4    786
3    707
1    681
2    679
0    624
Name: count, dtype: int64
모델 'monologg/koelectra-base-v3-discriminator' 로드 중...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

...

Validation Loss: 0.3483
Validation Macro F1: 0.8989
성능 향상 없음: 2/2

Early Stopping! 2번의 에폭 동안 성능 향상 없음.


평가 중: 100%|██████████| 24/24 [00:02<00:00,  8.63it/s]



테스트 결과:
Test Loss: 0.3493
Test Macro F1: 0.8913

분류 보고서:
              precision    recall  f1-score   support

       협박 대화       0.85      0.85      0.85       134
       갈취 대화       0.87      0.82      0.85       146
 직장 내 괴롭힘 대화       0.96      0.92      0.94       145
   기타 괴롭힘 대화       0.80      0.86      0.83       152
       일반 대화       0.99      0.99      0.99       169

    accuracy                           0.89       746
   macro avg       0.89      0.89      0.89       746
weighted avg       0.90      0.89      0.89       746

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package fonts-nanum.
(Reading database ... 126101 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...


/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 54801 (\N{HANGUL SYLLABLE HYEOB}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 48149 (\N{HANGUL SYLLABLE BAG}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 45824 (\N{HANGUL SYLLABLE DAE}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 54868 (\N{HANGUL SYLLABLE HWA}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 44040 (\N{HANGUL SYLLABLE GAL}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 52712 (\N{HANGUL SYLLABLE CWI}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/u

제출 파일 생성 완료: data/submission.csv


In [6]:
import os
import re
import numpy as np
import pandas as pd
from functools import partial
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizer, ElectraForSequenceClassification#, AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns

# 시드 고정
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True

set_seed(42)

#--------------------------
# 데이터 전처리 함수들
#--------------------------

# 1. 텍스트 정제 함수들
def remove_stopwords(tokens, stopword_list):
    stopword_set = stopword_list if isinstance(stopword_list, set) else set(stopword_list)
    return [token for token in tokens if token not in stopword_set]

def clean_text(text):
    text = re.sub(r"[^\w\s가-힣]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def normalize_repetitions(text, repeat_limit=2):
    # 문자 반복 (예: ㅋㅋㅋㅋ → ㅋㅋ)
    text = re.sub(r'(.)\1{2,}', lambda m: m.group(1) * repeat_limit, text)

    # 음절 반복 (예: 하하하하 → 하하)
    text = re.sub(r'((..))\\1{1,}', lambda m: m.group(1) * repeat_limit, text)

    return text

# 2. 텍스트 정제
def tokenize_and_clean_text(text, stopword_list=None, repeat_limit=2):
    text = normalize_repetitions(text, repeat_limit=repeat_limit)
    text = clean_text(text)
    tokens = text.split()
    if stopword_list:
        tokens = remove_stopwords(tokens, stopword_list)
    return tokens

# 3. 한줄 단위 전처리
def preprocess_conversation_lines(
    text,
    stopwords=None,
    use_silence=False,
    speaker_token="[UTTER]",
    repeat_limit=2
):
    lines = text.strip().split('\n')
    results = []

    for line in lines:
        if not line.strip():
            processed = ["[SILENCE]"] if use_silence else []
        else:
            processed = tokenize_and_clean_text(line, stopword_list=stopwords, repeat_limit=repeat_limit)
            if not processed:
                processed = ["[SILENCE]"] if use_silence else []
            else:
                processed = [speaker_token] + processed if speaker_token else processed

        if processed:
            results.append(" ".join(processed).strip())

    return results

# 4. 여러줄을 한 줄로 flatten 함수
def flatten_utterances(utterance_tokens_list, sep_token=" "):
    return sep_token.join(utterance_tokens_list).strip()

# 5. 전체 전처리 파이프라인
def preprocess(
    text,
    stopwords=None,
    speaker_token="[UTTER]",
    use_silence=True,
    sep_token=" ",
    repeat_limit=2
):
    """
    전체 전처리 통합 함수
    """
    utterance_tokens = preprocess_conversation_lines(
        text,
        stopwords=stopwords,
        use_silence=use_silence,
        speaker_token=speaker_token,
        repeat_limit=repeat_limit
    )
    return flatten_utterances(utterance_tokens, sep_token=sep_token)

# 결측치 및 중복 제거 함수
def clean_dataframe(df, is_train=True):
    df = df.dropna(subset=['text'])
    df = df.drop_duplicates(subset=['text'])
    if is_train:
        df = df.dropna(subset=['class'])
    df = df.reset_index(drop=True)
    return df

# CSV 파일 로드 함수
def load_csv_files(file_list, is_train=True):
    df_list = []

    for file_path in file_list:
        df = pd.read_csv(file_path)

        if is_train:
            if 'conversation' in df.columns:
                df = df.rename(columns={'conversation': 'text'})
        else:
            if 'conversation' in df.columns:
                df = df.rename(columns={'conversation': 'text'})
            if 'class' not in df.columns:
                df['class'] = pd.NA  # test에는 class가 없으므로 NaN 처리

        df_list.append(df)

    return pd.concat(df_list, ignore_index=True)

# 데이터프레임 준비 함수
def prepare_dataset(file_paths, preprocess_func, is_train=True):
    df = load_csv_files(file_paths, is_train=is_train)
    df = clean_dataframe(df, is_train=is_train)
    df['text'] = df['text'].apply(preprocess_func)
    df = clean_dataframe(df, is_train=is_train)
    return df

#--------------------------
# 모델 학습 설정
#--------------------------

# 설정값
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 10  # 5에서 10으로 변경
SCHEDULER_EPOCHS = 5  # 스케줄러용 에폭 (원래 설정 유지)
EARLY_STOPPING_PATIENCE = 2  # 연속 2번의 에폭 동안 성능이 향상되지 않으면 학습 중단
LEARNING_RATE = 2e-5
TRAIN_PATH = "data/train+normal_conversation.csv"
TEXT_COL = "text"#"conversation"x : 편리성 때문에
LABEL_COL = "class"
LABEL_DICT = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
ID_TO_LABEL = {v: k for k, v in LABEL_DICT.items()}
GPU_NUM = 0
TEST_FILE = "data/test.csv"

# 1. 데이터 전처리 및 로드
print("데이터 전처리 및 로드 중...")

# # 불용어 리스트 정의
# stopwords = '이 있 하 것 들 그 되 수 이 보 않 없 나 주 아니 등 같 우리 때 년 가 한 지 오 네 야 아 아니 그럼 내가 너'.split()
# 불용어 리스트 불러오기
stopwords = load_stopwords('enhanced_korean_stopwords.txt')

# 전처리 함수 설정 - 불용어 적용
preprocess_fn = partial(
    preprocess,
    stopwords=stopwords,   # 불용어 리스트 사용
    speaker_token="",      # 발화 단위 토큰 사용하지 않음
    sep_token=" ",         # 줄 구분 시 토큰
    use_silence=False,     # [SILENCE] 토큰 사용하지 않음
    repeat_limit=2         # 반복 문자 2개까지 허용
)

# 데이터셋 준비
train_files = [TRAIN_PATH]

# 데이터 전처리 및 저장
print("데이터 전처리 시작...")
train_df = prepare_dataset(train_files, preprocess_fn, is_train=True)
# train_df.to_csv("dataset/train_preprocessed_1.csv", index=False)
# print(f"✅ 전처리된 데이터 저장 완료: {train_df.shape}")
print(train_df.head())

# 라벨 매핑
train_df[LABEL_COL] = train_df[LABEL_COL].map(LABEL_DICT)

# 데이터 분할
train_data, temp_data = train_test_split(train_df, test_size=0.3, random_state=42, stratify=train_df[LABEL_COL])
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=temp_data[LABEL_COL])

print(f"학습 데이터 크기: {len(train_data)}")
print(f"검증 데이터 크기: {len(val_data)}")
print(f"테스트 데이터 크기: {len(test_data)}")

# 클래스별 데이터 개수 확인
print("학습 데이터의 클래스 분포:")
print(train_data[LABEL_COL].value_counts())

# 2. Dataset 클래스 정의
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 3. 데이터 로더 생성
def create_data_loader(df, tokenizer, max_len, batch_size):
    dataset = TextClassificationDataset(
        texts=df[TEXT_COL].values,
        labels=df[LABEL_COL].values,
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=2
    )

# 4. 토크나이저 및 모델 로드
print(f"모델 '{MODEL_NAME}' 로드 중...")
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)
model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_DICT))

# 5. 데이터 로더 생성
train_data_loader = create_data_loader(train_data, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(val_data, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_data, tokenizer, MAX_LEN, BATCH_SIZE)

# 6. 모델 학습 함수
def train_model(model, data_loader, optimizer, scheduler, device):
    model.train()
    losses = []

    progress_bar = tqdm(data_loader, desc="학습 중")

    for batch in progress_bar:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        losses.append(loss.item())
        progress_bar.set_postfix({"loss": f"{np.mean(losses):.4f}"})

    return np.mean(losses)

# 7. 모델 평가 함수
def evaluate_model(model, data_loader, device):
    model.eval()
    losses = []
    predictions = []
    real_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="평가 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
            )

            _, preds = torch.max(outputs.logits, dim=1)

            losses.append(outputs.loss.item())
            predictions.extend(preds.detach().cpu().numpy())
            real_labels.extend(labels.detach().cpu().numpy())

    return np.mean(losses), predictions, real_labels

# 8. 테스트 세트 예측 함수
def predict_test(model, data_loader, device):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="테스트 예측 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.detach().cpu().numpy())

    return predictions

# 9. 혼동 행렬 시각화 함수
def plot_confusion_matrix(cm, classes):
    # 한글 폰트 설정
    import matplotlib.font_manager as fm

    # 시스템에 설치된 폰트 경로 확인
    # 리눅스 환경이면 'NanumGothic'이나 다른 한글 폰트 사용
    # 윈도우라면 'Malgun Gothic' 등 사용
    try:
        # 방법 1: 나눔 글꼴 설치 및 사용
        !apt-get update -qq
        !apt-get install fonts-nanum -qq
        plt.rc('font', family='NanumGothic')
    except:
        try:
            # 방법 2: 한글 레이블을 영어로 변환
            classes = ['Threat', 'Extortion', 'Workplace Harassment', 'Other Harassment', 'Normal Conversation']
        except:
            pass

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('예측값')
    plt.ylabel('실제값')
    plt.title('혼동 행렬')
    plt.savefig('confusion_matrix.png')
    plt.close()

# 10. 학습 메인 함수
def train():
    # GPU 사용 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")
    print(f"사용 중인 디바이스: {device}")
    model.to(device)

    # 옵티마이저 및 스케줄러 설정
    # AdamW 임포트 부분 수정
    # from transformers import AdamW - 이 부분이 오류 발생
    from torch.optim import AdamW  # 이렇게 수정
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)  # correct_bias 파라미터 제거
    total_steps = len(train_data_loader) * SCHEDULER_EPOCHS  # 원래 에폭(5) 기준으로 설정

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # 학습 시작
    best_val_f1 = 0
    patience_counter = 0  # Early Stopping을 위한 카운터

    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch + 1}/{EPOCHS}")

        # 학습
        train_loss = train_model(model, train_data_loader, optimizer, scheduler, device)
        print(f"Train Loss: {train_loss:.4f}")

        # 검증
        val_loss, val_predictions, val_labels = evaluate_model(model, val_data_loader, device)
        val_f1 = f1_score(val_labels, val_predictions, average='macro')

        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Macro F1: {val_f1:.4f}")

        # 최고 성능 모델 저장 및 Early Stopping 로직
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"모델 저장됨: best_model.pth (F1: {val_f1:.4f})")
            patience_counter = 0  # 성능이 향상되었으므로 카운터 초기화
        else:
            patience_counter += 1  # 성능이 향상되지 않았으므로 카운터 증가
            print(f"성능 향상 없음: {patience_counter}/{EARLY_STOPPING_PATIENCE}")

            # Early Stopping 조건 확인
            if patience_counter >= EARLY_STOPPING_PATIENCE:
                print(f"\nEarly Stopping! {EARLY_STOPPING_PATIENCE}번의 에폭 동안 성능 향상 없음.")
                break

    # 최고 성능 모델 로드
    model.load_state_dict(torch.load("best_model.pth"))

    # 테스트 세트 평가
    test_loss, test_predictions, test_labels = evaluate_model(model, test_data_loader, device)
    test_f1 = f1_score(test_labels, test_predictions, average='macro')

    print("\n테스트 결과:")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Macro F1: {test_f1:.4f}")

    # 분류 보고서 출력
    class_names = [ID_TO_LABEL[i] for i in range(len(LABEL_DICT))]
    print("\n분류 보고서:")
    print(classification_report(test_labels, test_predictions, target_names=class_names))

    # 혼동 행렬 시각화
    cm = confusion_matrix(test_labels, test_predictions)
    plot_confusion_matrix(cm, class_names)

    return model

# 11. 실제 테스트 데이터 예측 및 제출 파일 생성
def predict_and_save():
    # 테스트 파일 로드
    test_data = pd.read_csv(TEST_FILE)

    # 텍스트 전처리 적용 - 학습 데이터와 동일한 방식으로 전처리
    # 줄 바꿈 표시 X, 불용어 X 설정
    test_data['processed_text'] = test_data['text'].apply(
        lambda x: preprocess(
            x,
            stopwords=stopwords,   # 불용어 리스트 사용
            speaker_token="",      # 발화 단위 토큰 사용하지 않음
            sep_token=" ",         # 줄 구분 시 토큰
            use_silence=False,     # [SILENCE] 토큰 사용하지 않음
            repeat_limit=2         # 반복 문자 2개까지 허용
        )
    )

    # Dataset 생성
    class TestDataset(Dataset):
        def __init__(self, texts, tokenizer, max_len):
            self.texts = texts
            self.tokenizer = tokenizer
            self.max_len = max_len

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, idx):
            text = str(self.texts[idx])

            encoding = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                return_token_type_ids=True,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )

            return {
                'text': text,
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'token_type_ids': encoding['token_type_ids'].flatten(),
            }

    # 테스트 데이터 로더 생성
    test_dataset = TestDataset(
        texts=test_data['processed_text'].values,
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )

    test_data_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=2
    )

    # GPU 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")

    # 최고 성능 모델 로드
    model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_DICT))
    model.load_state_dict(torch.load("best_model.pth"))
    model.to(device)

    # 예측
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(test_data_loader, desc="테스트 예측 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.detach().cpu().numpy())

    # 예측 결과 라벨로 변환 (숫자로 유지)
    label_predictions = [str(p) for p in predictions]

    # 제출 파일 생성
    submission = pd.DataFrame({
        'idx': test_data['idx'],
        'class': label_predictions
    })

    submission.to_csv('data/submission.csv', index=False)
    print(f"제출 파일 생성 완료: data/submission.csv")

# 메인 실행
if __name__ == "__main__":
    # 모델 학습
    model = train()

    # 테스트 데이터 예측 및 제출 파일 생성
    predict_and_save()

데이터 전처리 및 로드 중...
불용어 105개 로드 완료
데이터 전처리 시작...
   idx      class                                               text
0    0      협박 대화  스스로를 죽여달라고 애원하는 것인가 아닙니다 죄송합니다 죽을 거면 혼자 죽지 우리까...
1    1      협박 대화  길동경찰서입니다 9시 40분 마트에 폭발물을 설치할거다 똑바로 들어 한번만 얘기한다...
2    2  기타 괴롭힘 대화  되게 귀여운거 알지 나보다 작은 남자는 첨봤어 그만해 니들 놀리는거 재미없어 지영아...
3    3      갈취 대화  어이 말이야 이리 오라고 일 옷 좋아보인다 돈 있나봐 아니에요 돈 없어요 뒤져서 나...
4    4      갈취 대화  저기요 혹시 날이 너무 뜨겁잖아요 회사에서 선크림 파는데 한 번 손등에 발라보실래요...
학습 데이터 크기: 3477
검증 데이터 크기: 745
테스트 데이터 크기: 746
학습 데이터의 클래스 분포:
class
4    786
3    707
1    681
2    679
0    624
Name: count, dtype: int64
모델 'monologg/koelectra-base-v3-discriminator' 로드 중...


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


사용 중인 디바이스: cuda:0

Epoch 1/10


학습 중: 100%|██████████| 109/109 [00:36<00:00,  2.98it/s, loss=1.2065]


Train Loss: 1.2065


...

Validation Loss: 0.3413
Validation Macro F1: 0.8973
성능 향상 없음: 2/2

Early Stopping! 2번의 에폭 동안 성능 향상 없음.


평가 중: 100%|██████████| 24/24 [00:02<00:00,  8.55it/s]



테스트 결과:
Test Loss: 0.3520
Test Macro F1: 0.8953

분류 보고서:
              precision    recall  f1-score   support

       협박 대화       0.84      0.87      0.85       134
       갈취 대화       0.88      0.84      0.86       146
 직장 내 괴롭힘 대화       0.96      0.92      0.94       145
   기타 괴롭힘 대화       0.82      0.86      0.84       152
       일반 대화       0.99      0.99      0.99       169

    accuracy                           0.90       746
   macro avg       0.90      0.89      0.90       746
weighted avg       0.90      0.90      0.90       746

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 54801 (\N{HANGUL SYLLABLE HYEOB}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 48149 (\N{HANGUL SYLLABLE BAG}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 45824 (\N{HANGUL SYLLABLE DAE}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 54868 (\N{HANGUL SYLLABLE HWA}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 44040 (\N{HANGUL SYLLABLE GAL}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/usr/local/lib/python3.11/dist-packages/seaborn/utils.py:61: UserWarning: Glyph 52712 (\N{HANGUL SYLLABLE CWI}) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
/u

제출 파일 생성 완료: data/submission.csv
